# 用户冷启动-利用用户注册信息

In [ ]:
# 导入包
import random
import math
import numpy as np
import time
from tqdm import tqdm, trange

## 一. 通用函数定义

In [ ]:
# 定义装饰器，监控运行时间
def timmer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        res = func(*args, **kwargs)
        stop_time = time.time()
        print('Func %s, run time: %s' % (func.__name__, stop_time - start_time))
        return res
    return wrapper

### 1. 数据处理相关
1. load data
2. split data

In [ ]:
class Dataset():
    
    def __init__(self, fp, up):
        # fp: data file path
        # up: user profile path
        self.data, self.profile = self.loadData(fp, up)
    
    @timmer
    def loadData(self, fp, up):
        data = []
        for l in open(fp):
            data.append(tuple(l.strip().split('\t')[:2]))
        profile = {}
        for l in open(up):
            user, gender, age, country, _ = l.strip().split('\t')
            if age == '': 
                age = -1
            profile[user] = {'gender': gender, 'age': int(age), 'country': country}
        # 按照用户进行采样
        users = list(profile.keys())
        random.shuffle(users)
        users = set(users[:5000])
        data = [x for x in data if x[0] in users]
        profile = {k: profile[k] for k in users}
        return data, profile
    
    @timmer
    def splitData(self, M, k, seed=1):
        '''
        :params: data, 加载的所有(user, item)数据条目
        :params: M, 划分的数目，最后需要取M折的平均
        :params: k, 本次是第几次划分，k~[0, M)
        :params: seed, random的种子数，对于不同的k应设置成一样的
        :return: train, test
        '''
        train, test = [], []
        random.seed(seed)
        for user, item in self.data:
            # 这里与书中的不一致，本人认为取M-1较为合理，因randint是左右都覆盖的
            if random.randint(0, M-1) == k:  
                test.append((user, item))
            else:
                train.append((user, item))

        # 处理成字典的形式，user->set(items)
        def convert_dict(data):
            data_dict = {}
            for user, item in data:
                if user not in data_dict:
                    data_dict[user] = set()
                data_dict[user].add(item)
            data_dict = {k: list(data_dict[k]) for k in data_dict}
            return data_dict

        return convert_dict(train), convert_dict(test), self.profile

### 2. 评价指标
1. Precision
2. Recall
3. Coverage

In [ ]:
class Metric():
    
    def __init__(self, train, test, GetRecommendation):
        '''
        :params: train, 训练数据
        :params: test, 测试数据
        :params: GetRecommendation, 为某个用户获取推荐物品的接口函数
        '''
        self.train = train
        self.test = test
        self.GetRecommendation = GetRecommendation
        self.recs = self.getRec()
        
    # 为test中的每个用户进行推荐
    def getRec(self):
        recs = {}
        for user in self.test:
            rank = self.GetRecommendation(user)
            recs[user] = rank
        return recs
        
    # 定义精确率指标计算方式
    def precision(self):
        all, hit = 0, 0
        for user in self.test:
            test_items = set(self.test[user])
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1
            all += len(rank)
        return round(hit / all * 100, 2)
    
    # 定义召回率指标计算方式
    def recall(self):
        all, hit = 0, 0
        for user in self.test:
            test_items = set(self.test[user])
            rank = self.recs[user]
            for item, score in rank:
                if item in test_items:
                    hit += 1
            all += len(test_items)
        return round(hit / all * 100, 2)
    
    # 定义覆盖率指标计算方式
    def coverage(self):
        all_item, recom_item = set(), set()
        for user in self.test:
            if user in self.train:
                for item in self.train[user]:
                    all_item.add(item)
            rank = self.recs[user]
            for item, score in rank:
                recom_item.add(item)
        return round(len(recom_item) / len(all_item) * 100, 2)
    
    def eval(self):
        metric = {'Precision': self.precision(),
                  'Recall': self.recall(),
                  'Coverage': self.coverage()}
        print('Metric:', metric)
        return metric

## 二. 算法实现
1. MostPopular
2. GenderMostPopular
3. AgeMostPopular
4. CountryMostPopular
5. DemographicMostPopular

In [ ]:
# 1. MostPopular算法
def MostPopular(train, profile, N):
    '''
    :params: train, 训练数据
    :params: profile, 用户的注册信息
    :params: N, 推荐TopN物品的个数
    :return: GetRecommendation, 获取推荐结果的接口
    '''
    
    items = {}
    for user in train:
        for item in train[user]:
            if item not in items:
                items[item] = 0
            items[item] += 1
    items = list(sorted(items.items(), key=lambda x: x[1], reverse=True))
        
    # 获取接口函数
    def GetRecommendation(user):
        seen_items = set(train[user]) if user in train else set()
        recs = [x for x in items if x[0] not in seen_items][:N]
        return recs
    
    return GetRecommendation

In [ ]:
# 2. GenderMostPopular算法
def GenderMostPopular(train, profile, N):
    '''
    :params: train, 训练数据
    :params: profile, 用户的注册信息
    :params: N, 推荐TopN物品的个数
    :return: GetRecommendation, 获取推荐结果的接口
    '''
    
    mitems, fitems = {}, {} # 男、女
    for user in train:
        if profile[user]['gender'] == 'm':
            tmp = mitems
        elif profile[user]['gender'] == 'f':
            tmp = fitems
        for item in train[user]:
            if item not in tmp:
                tmp[item] = 0
            tmp[item] += 1
    mitems = list(sorted(mitems.items(), key=lambda x: x[1], reverse=True))
    fitems = list(sorted(fitems.items(), key=lambda x: x[1], reverse=True))
    
    mostPopular = MostPopular(train, profile, N)
    
    # 获取接口函数
    def GetRecommendation(user):
        seen_items = set(train[user]) if user in train else set()
        if profile[user]['gender'] == 'm':
            recs = [x for x in mitems if x[0] not in seen_items][:N]
        elif profile[user]['gender'] == 'f':
            recs = [x for x in fitems if x[0] not in seen_items][:N]
        else: # 没有提供性别信息的，按照MostPopular推荐
            recs = mostPopular(user)
        return recs
    
    return GetRecommendation

In [ ]:
# 3. AgeMostPopular算法
def AgeMostPopular(train, profile, N):
    '''
    :params: train, 训练数据
    :params: profile, 用户的注册信息
    :params: N, 推荐TopN物品的个数
    :return: GetRecommendation, 获取推荐结果的接口
    '''
    
    # 对年龄进行分段
    ages = []
    for user in profile:
        if profile[user]['age'] >= 0:
            ages.append(profile[user]['age'])
    maxAge, minAge = max(ages), min(ages)
    items = [{} for _ in range(int(maxAge // 10 + 1))]
    
    # 分年龄段进行统计
    for user in train:
        if profile[user]['age'] >= 0:
            age = profile[user]['age'] // 10
            for item in train[user]:
                if item not in items[age]:
                    items[age][item] = 0
                items[age][item] += 1
    for i in range(len(items)):
        items[i] = list(sorted(items[i].items(), key=lambda x: x[1], reverse=True))
    
    mostPopular = MostPopular(train, profile, N)
    
    # 获取接口函数
    def GetRecommendation(user):
        seen_items = set(train[user]) if user in train else set()
        if profile[user]['age'] >= 0:
            age = profile[user]['age'] // 10
            # 年龄信息异常的，按照全局推荐
            if age >= len(items) or len(items[age]) == 0:
                recs = mostPopular(user)
            else:
                recs = [x for x in items[age] if x[0] not in seen_items][:N]
        else: # 没有提供年龄信息的，按照全局推荐
            recs = mostPopular(user)
        return recs
    
    return GetRecommendation

In [ ]:
# 4. CountryMostPopular算法
def CountryMostPopular(train, profile, N):
    '''
    :params: train, 训练数据
    :params: profile, 用户的注册信息
    :params: N, 推荐TopN物品的个数
    :return: GetRecommendation, 获取推荐结果的接口
    '''
        
    # 分城市进行统计
    items = {}
    for user in train:
        country = profile[user]['country']
        if country not in items:
            items[country] = {}
        for item in train[user]:
            if item not in items[country]:
                items[country][item] = 0
            items[country][item] += 1
    for country in items:
        items[country] = list(sorted(items[country].items(), key=lambda x: x[1], reverse=True))
    
    mostPopular = MostPopular(train, profile, N)
    
    # 获取接口函数
    def GetRecommendation(user):
        seen_items = set(train[user]) if user in train else set()
        country = profile[user]['country']
        if country in items:
            recs = [x for x in items[country] if x[0] not in seen_items][:N]
        else: # 没有提供城市信息的，按照全局推荐
            recs = mostPopular(user)
        return recs
    
    return GetRecommendation

In [ ]:
# 5. DemographicMostPopular算法
def DemographicMostPopular(train, profile, N):
    '''
    :params: train, 训练数据
    :params: profile, 用户的注册信息
    :params: N, 推荐TopN物品的个数
    :return: GetRecommendation, 获取推荐结果的接口
    '''

    # 建立多重字典，将缺失值当成other，同归为一类进行处理
    items = {}
    for user in train:
        gender = profile[user]['gender']
        if gender not in items:
            items[gender] = {}
        age = profile[user]['age'] // 10
        if age not in items[gender]:
            items[gender][age] = {}
        country = profile[user]['country']
        if country not in items[gender][age]:
            items[gender][age][country] = {}
        for item in train[user]:
            if item not in items[gender][age][country]:
                items[gender][age][country][item] = 0
            items[gender][age][country][item] += 1
    for gender in items:
        for age in items[gender]:
            for country in items[gender][age]:
                items[gender][age][country] = list(sorted(items[gender][age][country].items(), 
                                                          key=lambda x: x[1], reverse=True))
                
    mostPopular = MostPopular(train, profile, N)
    
    # 获取接口函数
    def GetRecommendation(user):
        seen_items = set(train[user]) if user in train else set()
        gender = profile[user]['gender']
        age = profile[user]['age']
        country = profile[user]['country']
        if gender not in items or age not in items[gender] or country not in items[gender][age]:
            recs = mostPopular(user)
        else:
            recs = [x for x in items[gender][age][country] if x[0] not in seen_items][:N]
        return recs
    
    return GetRecommendation

## 三. 实验

采用Last.FM数据集，为加快实验速度，只取5000名用户的记录进行实验

1. MostPopular实验
2. GenderMostPopular实验
3. AgeMostPopular实验
4. CountryMostPopular实验
5. DemographicMostPopular实验

M=10, N=10

In [ ]:
class Experiment():
    
    def __init__(self, M, N, at='MostPopular',
                 fp='../dataset/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv',
                 up='../dataset/lastfm-dataset-360K/usersha1-profile.tsv'):
        '''
        :params: M, 进行多少次实验
        :params: N, TopN推荐物品的个数
        :params: fp, 数据文件路径
        :params: up, 用户注册信息文件路径
        '''
        self.M = M
        self.N = N
        self.fp = fp
        self.up = up
        self.at = at
        self.alg = {'MostPopular': MostPopular, 'GenderMostPopular': GenderMostPopular,
                    'AgeMostPopular': AgeMostPopular, 'CountryMostPopular': CountryMostPopular,
                    'DemographicMostPopular': DemographicMostPopular}
    
    # 定义单次实验
    @timmer
    def worker(self, train, test, profile):
        '''
        :params: train, 训练数据集
        :params: test, 测试数据集
        :params: profile, 用户注册信息
        :return: 各指标的值
        '''
        getRecommendation = self.alg[self.at](train, profile, self.N)
        metric = Metric(train, test, getRecommendation)
        return metric.eval()
    
    # 多次实验取平均
    @timmer
    def run(self):
        metrics = {'Precision': 0, 'Recall': 0, 
                   'Coverage': 0}
        dataset = Dataset(self.fp, self.up)
        for ii in range(self.M):
            train, test, profile = dataset.splitData(self.M, ii)
            print('Experiment {}:'.format(ii))
            metric = self.worker(train, test, profile)
            metrics = {k: metrics[k]+metric[k] for k in metrics}
        metrics = {k: metrics[k] / self.M for k in metrics}
        print('Average Result (M={}, N={}): {}'.format(\
                              self.M, self.N, metrics))

In [ ]:
# 1. MostPopular实验
M, N = 10, 10
exp = Experiment(M, N, at='MostPopular')
exp.run()

In [ ]:
# 2. GenderMostPopular实验
M, N = 10, 10
exp = Experiment(M, N, at='GenderMostPopular')
exp.run()

In [ ]:
# 3. AgeMostPopular实验
M, N = 10, 10
exp = Experiment(M, N, at='AgeMostPopular')
exp.run()

In [ ]:
# 4. CountryMostPopular实验
M, N = 10, 10
exp = Experiment(M, N, at='CountryMostPopular')
exp.run()

In [ ]:
# 5. DemographicMostPopular实验
M, N = 10, 10
exp = Experiment(M, N, at='DemographicMostPopular')
exp.run()

## 四. 实验结果

1. MostPopular实验

    Running time: 340.1061339378357
    
    Average Result (M=10, N=10): {'Precision': 2.28, 'Recall': 4.635, 'Coverage': 0.071}

2. GenderMostPopular实验

    Running time: 265.6223392486572
    
    Average Result (M=10, N=10): {'Precision': 2.326, 'Recall': 4.727, 'Coverage': 0.107}

3. AgeMostPopular实验

    Running time: 205.56471586227417
    
    Average Result (M=10, N=10): {'Precision': 2.379, 'Recall': 4.822, 'Coverage': 0.455}
    
4. CountryMostPopular实验

    Running time: 69.32589602470398
    
    Average Result (M=10, N=10): {'Precision': 2.415, 'Recall': 4.855, 'Coverage': 2.5}

5. DemographicMostPopular实验

    Running time: 73.98487401008606
    
    Average Result (M=10, N=10): {'Precision': 2.42, 'Recall': 4.86, 'Coverage': 2.5}

## 附录：运行日志(请双击查看)

1. MostPopular实验
Func loadData, run time: 31.807884693145752
Func splitData, run time: 0.6068239212036133
Experiment 0:
Metric: {'Precision': 2.27, 'Recall': 4.61, 'Coverage': 0.08}
Func worker, run time: 28.971026182174683
Func splitData, run time: 0.44267868995666504
Experiment 1:
Metric: {'Precision': 2.26, 'Recall': 4.59, 'Coverage': 0.07}
Func worker, run time: 28.366169691085815
Func splitData, run time: 0.5044941902160645
Experiment 2:
Metric: {'Precision': 2.22, 'Recall': 4.5, 'Coverage': 0.07}
Func worker, run time: 38.45435905456543
Func splitData, run time: 0.4402909278869629
Experiment 3:
Metric: {'Precision': 2.31, 'Recall': 4.67, 'Coverage': 0.07}
Func worker, run time: 28.688590049743652
Func splitData, run time: 0.48880600929260254
Experiment 4:
Metric: {'Precision': 2.33, 'Recall': 4.77, 'Coverage': 0.07}
Func worker, run time: 28.041622161865234
Func splitData, run time: 0.5104248523712158
Experiment 5:
Metric: {'Precision': 2.2, 'Recall': 4.51, 'Coverage': 0.07}
Func worker, run time: 28.33553171157837
Func splitData, run time: 0.5063672065734863
Experiment 6:
Metric: {'Precision': 2.33, 'Recall': 4.72, 'Coverage': 0.07}
Func worker, run time: 29.89447808265686
Func splitData, run time: 0.4293487071990967
Experiment 7:
Metric: {'Precision': 2.24, 'Recall': 4.54, 'Coverage': 0.07}
Func worker, run time: 29.959038019180298
Func splitData, run time: 0.5041751861572266
Experiment 8:
Metric: {'Precision': 2.34, 'Recall': 4.71, 'Coverage': 0.07}
Func worker, run time: 30.90305995941162
Func splitData, run time: 0.4769570827484131
Experiment 9:
Metric: {'Precision': 2.3, 'Recall': 4.73, 'Coverage': 0.07}
Func worker, run time: 31.461721897125244
Average Result (M=10, N=10): {'Precision': 2.2800000000000002, 'Recall': 4.635, 'Coverage': 0.07100000000000002}
Func run, run time: 340.1061339378357

2. GenderMostPopular实验
Func loadData, run time: 32.967588901519775
Func splitData, run time: 0.45601916313171387
Experiment 0:
Metric: {'Precision': 2.32, 'Recall': 4.72, 'Coverage': 0.11}
Func worker, run time: 19.60267210006714
Func splitData, run time: 0.5031688213348389
Experiment 1:
Metric: {'Precision': 2.33, 'Recall': 4.73, 'Coverage': 0.1}
Func worker, run time: 25.32190990447998
Func splitData, run time: 0.42341017723083496
Experiment 2:
Metric: {'Precision': 2.38, 'Recall': 4.82, 'Coverage': 0.1}
Func worker, run time: 21.993903875350952
Func splitData, run time: 0.505748987197876
Experiment 3:
Metric: {'Precision': 2.4, 'Recall': 4.86, 'Coverage': 0.1}
Func worker, run time: 25.69679093360901
Func splitData, run time: 0.5266520977020264
Experiment 4:
Metric: {'Precision': 2.22, 'Recall': 4.54, 'Coverage': 0.11}
Func worker, run time: 21.68884301185608
Func splitData, run time: 0.5205812454223633
Experiment 5:
Metric: {'Precision': 2.31, 'Recall': 4.74, 'Coverage': 0.11}
Func worker, run time: 24.315038204193115
Func splitData, run time: 0.6323270797729492
Experiment 6:
Metric: {'Precision': 2.33, 'Recall': 4.7, 'Coverage': 0.11}
Func worker, run time: 24.59072780609131
Func splitData, run time: 0.5137548446655273
Experiment 7:
Metric: {'Precision': 2.35, 'Recall': 4.76, 'Coverage': 0.11}
Func worker, run time: 21.767657995224
Func splitData, run time: 0.5112850666046143
Experiment 8:
Metric: {'Precision': 2.3, 'Recall': 4.64, 'Coverage': 0.11}
Func worker, run time: 21.0389301776886
Func splitData, run time: 0.5135531425476074
Experiment 9:
Metric: {'Precision': 2.32, 'Recall': 4.76, 'Coverage': 0.11}
Func worker, run time: 21.189006090164185
Average Result (M=10, N=10): {'Precision': 2.326, 'Recall': 4.726999999999999, 'Coverage': 0.10700000000000001}
Func run, run time: 265.6223392486572

3. AgeMostPopular实验
Func loadData, run time: 30.702067852020264
Func splitData, run time: 3.189789295196533
Experiment 0:
Metric: {'Precision': 2.34, 'Recall': 4.75, 'Coverage': 0.45}
Func worker, run time: 14.813374996185303
Func splitData, run time: 0.5000729560852051
Experiment 1:
Metric: {'Precision': 2.36, 'Recall': 4.78, 'Coverage': 0.46}
Func worker, run time: 18.69653296470642
Func splitData, run time: 0.714263916015625
Experiment 2:
Metric: {'Precision': 2.5, 'Recall': 5.06, 'Coverage': 0.46}
Func worker, run time: 16.40371298789978
Func splitData, run time: 0.42107224464416504
Experiment 3:
Metric: {'Precision': 2.41, 'Recall': 4.86, 'Coverage': 0.47}
Func worker, run time: 16.157140016555786
Func splitData, run time: 0.44391894340515137
Experiment 4:
Metric: {'Precision': 2.32, 'Recall': 4.73, 'Coverage': 0.45}
Func worker, run time: 16.057992935180664
Func splitData, run time: 0.4891808032989502
Experiment 5:
Metric: {'Precision': 2.33, 'Recall': 4.75, 'Coverage': 0.45}
Func worker, run time: 16.826287746429443
Func splitData, run time: 0.5030360221862793
Experiment 6:
Metric: {'Precision': 2.38, 'Recall': 4.8, 'Coverage': 0.46}
Func worker, run time: 15.631379127502441
Func splitData, run time: 0.508782148361206
Experiment 7:
Metric: {'Precision': 2.32, 'Recall': 4.7, 'Coverage': 0.45}
Func worker, run time: 15.04989218711853
Func splitData, run time: 0.4729750156402588
Experiment 8:
Metric: {'Precision': 2.5, 'Recall': 5.02, 'Coverage': 0.46}
Func worker, run time: 19.40653085708618
Func splitData, run time: 0.4343280792236328
Experiment 9:
Metric: {'Precision': 2.33, 'Recall': 4.77, 'Coverage': 0.44}
Func worker, run time: 17.756699085235596
Average Result (M=10, N=10): {'Precision': 2.379, 'Recall': 4.822, 'Coverage': 0.45500000000000007}
Func run, run time: 205.56471586227417

4. CountryMostPopular实验
Func loadData, run time: 30.24172306060791
Func splitData, run time: 0.4623088836669922
Experiment 0:
Metric: {'Precision': 2.41, 'Recall': 4.85, 'Coverage': 2.53}
Func worker, run time: 3.4046268463134766
Func splitData, run time: 0.43274497985839844
Experiment 1:
Metric: {'Precision': 2.39, 'Recall': 4.79, 'Coverage': 2.49}
Func worker, run time: 3.3446362018585205
Func splitData, run time: 0.5451531410217285
Experiment 2:
Metric: {'Precision': 2.35, 'Recall': 4.72, 'Coverage': 2.52}
Func worker, run time: 3.223905086517334
Func splitData, run time: 0.4849269390106201
Experiment 3:
Metric: {'Precision': 2.39, 'Recall': 4.76, 'Coverage': 2.5}
Func worker, run time: 3.1288833618164062
Func splitData, run time: 0.44620418548583984
Experiment 4:
Metric: {'Precision': 2.4, 'Recall': 4.86, 'Coverage': 2.48}
Func worker, run time: 3.3267481327056885
Func splitData, run time: 0.4455292224884033
Experiment 5:
Metric: {'Precision': 2.4, 'Recall': 4.86, 'Coverage': 2.53}
Func worker, run time: 3.6315438747406006
Func splitData, run time: 0.5009069442749023
Experiment 6:
Metric: {'Precision': 2.38, 'Recall': 4.76, 'Coverage': 2.5}
Func worker, run time: 4.013977766036987
Func splitData, run time: 0.5759139060974121
Experiment 7:
Metric: {'Precision': 2.47, 'Recall': 4.96, 'Coverage': 2.48}
Func worker, run time: 3.3212859630584717
Func splitData, run time: 0.4942488670349121
Experiment 8:
Metric: {'Precision': 2.53, 'Recall': 5.05, 'Coverage': 2.48}
Func worker, run time: 3.3335418701171875
Func splitData, run time: 0.43490123748779297
Experiment 9:
Metric: {'Precision': 2.43, 'Recall': 4.94, 'Coverage': 2.49}
Func worker, run time: 3.2024052143096924
Average Result (M=10, N=10): {'Precision': 2.415, 'Recall': 4.854999999999999, 'Coverage': 2.5}
Func run, run time: 69.32589602470398

5. DemographicMostPopular实验
Func loadData, run time: 31.048221826553345
Func splitData, run time: 0.6405370235443115
Experiment 0:
Metric: {'Precision': 2.41, 'Recall': 4.85, 'Coverage': 2.53}
Func worker, run time: 3.8784639835357666
Func splitData, run time: 0.5229880809783936
Experiment 1:
Metric: {'Precision': 2.39, 'Recall': 4.79, 'Coverage': 2.49}
Func worker, run time: 3.4125118255615234
Func splitData, run time: 0.43076014518737793
Experiment 2:
Metric: {'Precision': 2.35, 'Recall': 4.72, 'Coverage': 2.52}
Func worker, run time: 3.470226287841797
Func splitData, run time: 0.4595067501068115
Experiment 3:
Metric: {'Precision': 2.39, 'Recall': 4.76, 'Coverage': 2.5}
Func worker, run time: 3.9592530727386475
Func splitData, run time: 0.5331680774688721
Experiment 4:
Metric: {'Precision': 2.4, 'Recall': 4.86, 'Coverage': 2.48}
Func worker, run time: 4.06541109085083
Func splitData, run time: 0.5062789916992188
Experiment 5:
Metric: {'Precision': 2.4, 'Recall': 4.86, 'Coverage': 2.53}
Func worker, run time: 3.6064438819885254
Func splitData, run time: 0.4431419372558594
Experiment 6:
Metric: {'Precision': 2.38, 'Recall': 4.76, 'Coverage': 2.5}
Func worker, run time: 3.7960410118103027
Func splitData, run time: 0.5151360034942627
Experiment 7:
Metric: {'Precision': 2.47, 'Recall': 4.96, 'Coverage': 2.48}
Func worker, run time: 4.031769037246704
Func splitData, run time: 0.4504380226135254
Experiment 8:
Metric: {'Precision': 2.53, 'Recall': 5.05, 'Coverage': 2.48}
Func worker, run time: 3.66860294342041
Func splitData, run time: 0.5212371349334717
Experiment 9:
Metric: {'Precision': 2.43, 'Recall': 4.94, 'Coverage': 2.49}
Func worker, run time: 3.6845691204071045
Average Result (M=10, N=10): {'Precision': 2.415, 'Recall': 4.854999999999999, 'Coverage': 2.5}
Func run, run time: 73.98487401008606